This is the Logiq Tower puzzle. I received it as a Christmas gift at least three years ago, and have not yet found all 22,069 solutions. I have no idea what I've been doing with my life to prevent me from finding them all, but that gets fixed today. Of course finding all the solutions by hand is not practical, so I will rely on code to do it. The puzzle is an interesting take on the exact cover class of problems, where the space to cover has been wrapped in a cylinder around a central pillar. This makes the solutions modulo, in the sense that a piece sticking out of the board on the left or right will overflow into the other side. Think of the game asteroids; it's sort of like when your ship flies off one side of the screen and appears on the opposite one. The only difference is that the Logiq Tower space only circles around on the left and right hand sides but not the top. It's a cylindrical space rather than a toroidal one.

<img src='img/asteroids.gif' width='800px'>

There are fifteen different pieces that the puzzle makers have conveniently named for us:

<img src='img/pieces.JPG' width='500px'>

The puzzle is two layers thick, and twelve sectors wide. The height varies from two to four units tall. There are two different classes of pieces. The ones with numeric names exist in both layers, while the alphabetically named pieces exist only in the outer layer. The numeric pieces also always occupy the entire inner ring of the row they occupy, and so it's sufficient to only capture the spots they fill in the outer ring, and the row that they occupy. This is convenient since it means we can capture the problem in a two dimensional 12 x tower_height matrix, where height = 2, 3, 4, 5. For example, the sample solution provided for the two unit tall tower by the makers of the puzzle is:

In [1]:
import numpy as np

example_soln = np.array([['Q','Q','U','U','U','L','L','L','L','Y','0','0'],
                         ['Q','Q','U','2','U','L','2','Y','Y','Y','Y','Q']])
example_soln

array([['Q', 'Q', 'U', 'U', 'U', 'L', 'L', 'L', 'L', 'Y', '0', '0'],
       ['Q', 'Q', 'U', '2', 'U', 'L', '2', 'Y', 'Y', 'Y', 'Y', 'Q']],
      dtype='<U1')

To illustrate the modulo nature of the solution, we can shift the above solution by one to the right:

In [2]:
np.roll(example_soln, 1, axis=1)

array([['0', 'Q', 'Q', 'U', 'U', 'U', 'L', 'L', 'L', 'L', 'Y', '0'],
       ['Q', 'Q', 'Q', 'U', '2', 'U', 'L', '2', 'Y', 'Y', 'Y', 'Y']],
      dtype='<U1')

We will need to account for all of these redundant solutions at the end so we don't end up with more than the 22,069 solutions advertised. We can worry more about this when we get to the point of actually generating solutions.

The way dlx works, we will need to create a row for every possible location and orientation of each puzzle piece. This row will need to track which spots of the grid the piece occupies, as well as which pieces have been used. We can start by creating a basic representation of each piece in two dimensions:

In [3]:
I = np.array(['I','I','I','I','I']).reshape(1,-1)

Y = np.array([['.','.','Y','.'],
              ['Y','Y','Y','Y']])

L = np.array([['.','.','.','L'],
              ['L','L','L','L']])

N = np.array([['.','.','N','N'],
              ['N','N','N','.']])

U = np.array([['U','.','U'],
              ['U','U','U']])

Q = np.array([['.','Q','Q'],
              ['Q','Q','Q']])

W = np.array([['.','.','W'],
              ['.','W','W'],
              ['W','W','.']])

F = np.array([['.','F','.'],
              ['F','F','F'],
              ['F','.','.']])

T = np.array([['T','T','T'],
              ['.','T','.'],
              ['.','T','.']])

S = np.array([['.','S','S'],
              ['.','S','.'],
              ['S','S','.']])

zero = np.array(['0','0']).reshape(1,-1)

one = np.array(['1','.','1']).reshape(1,-1)

two = np.array(['2','.','.','2']).reshape(1,-1)

three = np.array(['3','.','.','.','3']).reshape(1,-1)

four = np.array(['4','.','.','.','.','4']).reshape(1,-1)

Some of the pieces also have flipped versions that are not equivalent when they are rotated 180 degrees. This applies to Y, L, N, U, Q, W, F, and T. We need to account for these upside down orientations:

In [4]:
Y_ud = np.rot90(Y, k=2)

L_ud = np.rot90(L, k=2)

N_ud = np.rot90(N, k=2)

U_ud = np.rot90(U, k=2)

Q_ud = np.rot90(Q, k=2)

W_ud = np.rot90(W, k=2)

F_ud = np.rot90(F, k=2)

T_ud = np.rot90(T, k=2)

To keep track of all of these pieces in their different orientations, we can store them in a dictionary of lists. The key will be the alphabetic or numeric name for the piece. The list will contain the possible orientations of the pieces:

In [5]:
piece_list = ['I', 'Y', 'L', 'N', 'U', 'Q', 'W', 'F', 
              'T', 'S', '0', '1', '2', '3', '4']

inner_layer = {'I':False,
               'Y':False,
               'L':False,
               'N':False,
               'U':False,
               'Q':False,
               'W':False,
               'F':False,
               'T':False,
               'S':False,
               '0':True,
               '1':True,
               '2':True,
               '3':True,
               '4':True}

symmetric = {'I':True,
             'Y':False,
             'L':False,
             'N':False,
             'U':False,
             'Q':False,
             'W':False,
             'F':False,
             'T':False,
             'S':True,
             '0':True,
             '1':True,
             '2':True,
             '3':True,
             '4':True}

piece_dict = {'I':[I],
              'Y':[Y],#,Y_ud],
              'L':[L],#,L_ud],
              'N':[N],#,N_ud],
              'U':[U],#,U_ud],
              'Q':[Q],#,Q_ud],
              'W':[W],#,W_ud],
              'F':[F],#,F_ud],
              'T':[T],#,T_ud],
              'S':[S],
              '0':[zero],
              '1':[one],
              '2':[two],
              '3':[three],
              '4':[four]}

Next we need to account for all the possible locations in the 12 x tower_height grid of each piece in each orientation:

In [6]:
def possible_piece_positions(tower_height, tower_width):
    blank_tower = np.zeros((tower_height,tower_width), 'U1')
    blank_tower.fill('.')
    piece_positions = {}
    for piece_name, orientations in piece_dict.items():
        piece_height = orientations[0].shape[0]
        piece_width = orientations[0].shape[1]
        
        # Skip this piece if it can't fit in the tower
        if orientations[0].shape[0] > tower_height:
            continue
        
        positions = []
        for orientation in orientations:
            piece = blank_tower.copy()
            piece[:piece_height, :piece_width] = orientation
            
            # For all the ways the piece fits vertically    
            for i in range(tower_height - piece_height + 1):
                # Since there can only be one numeric piece per row,
                # anchor it at the left side of the first row to reduce
                # the size of the problem space by 12.
                if i == 0 and inner_layer[piece_name]:
                    position = piece
                    positions.append(piece)
                    continue                    
                # For all of the horizontal positions it can be in
                for j in range(tower_width):
                    position = np.roll(np.roll(piece, j, axis=1), i, axis=0)
                    positions.append(position)
        
        piece_positions[piece_name] = positions

    return piece_positions

Next we need to take all of these piece positions and turn them into something the dlx solver code can work with. I wrote code to solve the dlx algorithm in the language go, but since I wanted to make use of numpy for ease of setup I decided against making this a go project early on. I fought off the "not invented here" syndrome and resisted the urge to re-write the dlx algorithm in yet another language. Instead I will use the implementation found here https://kunigami.blog/2013/04/28/the-algorithm-x-and-the-dancing-links/. I have modified it slightly to ensure that it returns all possible exact cover solutions rather than terminating after it finds the first solution. 

In [7]:
class Node:
    def __init__(self, row, col):
        self.row, self.col = row, col

    def deattach(self):
        self.up.down = self.down
        self.down.up = self.up

    def attach(self):
        self.down.up = self.up.down = self

class Head:
    def __init__(self, col):
        self.col = col

    def deattach(self):
        self.left.right = self.right
        self.right.left = self.left

    def attach(self):
        self.right.left = self.left.right = self

class NodeIterator:

    def __init__(self, node):
        self.curr = self.start = node

    def __iter__(self):
        return self

    def __next__(self):
        _next = self.move(self.curr)
        if _next == self.start:
            raise StopIteration
        else:
            self.curr = _next
            return _next

    def move(self):
        raise NotImplementedError

class LeftIterator (NodeIterator):
    def move(self, node):
        return node.left

class RightIterator (NodeIterator):
    def move(self, node):
        return node.right

class DownIterator (NodeIterator):
    def move(self, node):
        return node.down

class UpIterator (NodeIterator):
    def move(self, node):
        return node.up

class SparseMatrix:

    def createLeftRightLinks(self, srows):
        for srow in srows:
            n = len(srow)
            for j in range(n):
                srow[j].right = srow[(j + 1) % n]
                srow[j].left = srow[(j - 1 + n) % n]

    def createUpDownLinks(self, scols):
        for scol in scols:
            n = len(scol)
            for i in range(n):
                scol[i].down = scol[(i + 1) % n]
                scol[i].up = scol[(i - 1 + n) % n]
                scol[i].head = scol[0]

    def __init__(self, mat):

        nrows = len(mat)
        ncols = len(mat[0])

        srow = [[ ] for _ in range(nrows)]
        heads = [Head(j) for j in range(ncols)]
        scol = [[head] for head in heads]

        # Head of the column heads
        self.head = Head(-1)
        heads = [self.head] + heads

        self.createLeftRightLinks([heads])

        for i in range(nrows):
            for j in range(ncols):
                if mat[i][j] == 1:
                    node = Node(i, j)
                    scol[j].append(node)
                    srow[i].append(node)

        self.createLeftRightLinks(srow)
        self.createUpDownLinks(scol)

class DancingLinks:

    def __init__(self, mat):
        self.solution = [ ]
        self.smat = SparseMatrix(mat)

    def cover(self, col):
        col.deattach()
        for row in DownIterator(col):
            for cell in RightIterator(row):
                cell.deattach()

    def uncover(self, col):
        for row in UpIterator(col):
            for cell in LeftIterator(row):
                cell.attach()
        col.attach()

    def solve(self):
        solutions = []
        for s in self.backtrack():
            #print(sorted(s))
            solutions.append(sorted(s))
        return solutions

    def backtrack(self):
        # Let's cover the first uncovered item
        col = self.smat.head.right
        # No column left
        if col == self.smat.head:
            yield self.solution.copy()
            return
        # No set to cover this element
        if col.down == col:
            return

        self.cover(col)

        for row in DownIterator(col):

            for cell in RightIterator(row):
                self.cover(cell.head)
                
            self.solution.append(row.row)
            yield from self.backtrack()
            self.solution.pop()

            for cell in LeftIterator(row):
                self.uncover(cell.head)
        self.uncover(col)
        
        return

To provide some intuition for how the above solver works, it takes a column with many rows, each partially covering the set of columns. The job of the solver is to find all possible subsets of rows that exactly cover the columns. A solution cannot leave any column blank, and it cannot cover a column with more than one row. The following example of a very small problem makes it easy to see that the solver works as advertised.

In [8]:
demo_mat = [[0, 0, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1]]

demo_solver = DancingLinks(demo_mat)
demo_solutions = demo_solver.solve()
print(demo_solutions)

[[0, 3, 4], [5, 6], [7]]


Since the solver expects a binary matrix like above, we need to find a way to represent our puzzle in such a way. To do this each row needs to represents a piece in a given location. The columns will encode any key components of the puzzle covered by that piece. For this puzzle there are really only two things we need to track cover on; which pieces have been used (we cannot reuse the same piece in a different location), and which coordinates of the tower have been covered by the row in question. The first fifteen columns can represent which piece we have used, where the first is piece 'I' and the fifteen is piece '4'. Obviously each row should only cover one of these columns. We can represent the second aspect of the cover problem by flattening the 12 x tower_height 2D array into a 12 * tower_height 1D array and then simply convert all '.' empty values to 0's and all piece markers to 1's.

If we create a row for every piece produced by the possible_piece_positions() function, we'll almost have everything we need to start producing our first Logiq Tower solutions. The only part that is missing is to add the possibility of not using each piece. This is crucial to the 2, 3, and 4 unit tall towers which will not make use of all fifteen pieces. If we did not account for the left out pieces, it would be impossible to find an exact cover in the above formulation.

With this in mind, lets create a function to construct the dlx binary matrix for a tower of a given height using the output of possible_piece_positions():

In [9]:
def construct_dlx_matrix(tower_height, tower_width, symmetry_breaking_piece):
    piece_positions = possible_piece_positions(tower_height, tower_width)
    dlx_matrix = []
    
    # Create rows for each piece position
    for piece_name, positions in piece_positions.items():
        piece_cover = [0] * 15
        piece_cover[piece_list.index(piece_name)] = 1
        # If we ignore the symmetry of one of the symmetric pieces it will
        # break the symmetry of flipping the entire tower upside down for
        # all solution variants that include that piece.
        if piece_name == symmetry_breaking_piece:
            positions = [positions[0]]
        for p in positions: 
            inner_row_cover = [0] * tower_height
            if inner_layer[piece_name]:
                inner_row_index = np.where((p == piece_name).sum(axis=1) > 0)
                inner_row_index = inner_row_index[0][0]
                inner_row_cover[inner_row_index] = 1
            p_flat = p.flatten()
            row = np.zeros(p_flat.shape, dtype=int)
            row[np.where(p_flat == piece_name)] = 1
            row = piece_cover + inner_row_cover + row.tolist()
            dlx_matrix.append(row)
    
    # Create rows for each unused piece
    for i, piece_name in enumerate(piece_list):
        piece_cover = [0] * 15
        piece_cover[i] = 1
        inner_row_cover = [0] * tower_height
        row = piece_cover + inner_row_cover + [0] * tower_height * tower_width
        dlx_matrix.append(row)
    
    return dlx_matrix


def write_dlx_matrix_to_txt(fname, matrix):
    text_matrix = np.array(matrix, dtype=int)
    np.savetxt(fname, text_matrix, fmt='%i')

In [10]:
from time import time

solution_times = {}

tower_height = 2
tower_width = 12

start = time()

mat = construct_dlx_matrix(tower_height, tower_width, 'U')
#write_dlx_matrix_to_txt('tower_two.txt', mat)
solver = DancingLinks(mat)
row_list_solutions = solver.solve()
print('total solutions: ', len(row_list_solutions))

stop = time()
solution_times[tower_height] = stop - start
print('time (minutes): ', solution_times[2] / 60)

total solutions:  0
time (minutes):  0.10456072489420573


Five hunded and fifty two solutions were found in just over four minutes on my not so impressve laptop. This is more than the twenty three we expected, but keep in mind that there should be eleven more equal solutions that are simple rotational translations of this one. There are also twelve more that are equivalent to taking the puzzle matrix, rotating it by one hundred and eighty degrees, and then translating it through it's twelve equivalent solutions. So after dividing five hundred and fifty two by twenty four, we should have found twenty three unique solutions, exactly as expected.

To confirm this I will track the duplicate solutions with a dictionary that maps them all back to the first variant of that solution found. I will also store each of those first variant solutions in a separate list. To achive this it is necessary to write some helper functions that reconstuct the numpy array representation of a solution from the dlx_matrix rows returned by solve() on an instance of DancingLinks. First, it will be good to reconstruct the list of dlx_matrix rows into a numpy array that we can recognize as a solution:

In [11]:
def reconstruct_solution(row_list_solution, dlx_matrix, tower_height, tower_width):
    solution = np.zeros((tower_height,tower_width), 'U1')
    solution.fill('.')
    for row_index in row_list_solution:
        row = dlx_matrix[row_index]
        piece_name = piece_list[row[:15].index(1)]
        piece_position = row[15 + tower_height:]
        piece_position = np.array(piece_position)
        piece_position = piece_position.reshape(tower_height, tower_width)
        solution[np.where(piece_position == 1)] = piece_name
        
    return solution

In [13]:
reconstruct_solution(row_list_solutions[0], mat, tower_height, tower_width)

IndexError: list index out of range

Next, we will need to create the 23 additional equivalent solutions to the one we just found, and usethose to filter our full solution list down to just the unique ones.

In [14]:
def create_equivalent_solutions(solution_np_array):
    tower_width = solution_np_array.shape[1]
    equivalent_solutions = []
    for i in range(tower_width):
        eq_sol = np.roll(solution_np_array, i, axis=1)
        flipped_eq_sol = np.rot90(eq_sol, k=2)
        equivalent_solutions.append(eq_sol)
        equivalent_solutions.append(flipped_eq_sol)
        
    return equivalent_solutions
    

def find_unique_solutions(row_list_solutions, dlx_matrix, tower_height, tower_width):
    solutions_dict = {}
    unique_solution_list = []
    for row_list_solution in row_list_solutions:
        sol = reconstruct_solution(row_list_solution, mat, tower_height, tower_width)
        if sol.tostring() not in solutions_dict:
            equivalent_solutions = create_equivalent_solutions(sol)
            for eq_sol in equivalent_solutions:
                solutions_dict[eq_sol.tostring()] = sol
            unique_solution_list.append(sol)
            
    return solutions_dict, unique_solution_list

In [15]:
solution_dict, solution_list = find_unique_solutions(row_list_solutions, mat, tower_height, tower_width)
print(len(solution_list))

0


As predicted, the dancing links solver found all the possible orientations of the twenty three unique solutions for our two row tall Logiq Tower. Twenty three solutions isn't too many, so we can inspect all of them individually.

In [ ]:
for sol in solution_list:
    print(sol)
    print()

Now we can repet this exercise for the towers that are three, four, and five rows tall:

In [ ]:
full_solutions_dict = {}
full_solutions_dict[2] = solution_list

for th in range(3,6):
    tower_height = th
    
    start = time()
    
    mat = construct_dlx_matrix(tower_height, tower_width, 'U')
    solver = DancingLinks(mat)
    solutions = solver.solve()
    
    _, solution_list = find_unique_solutions(solutions, mat, tower_height, tower_width)
    
    full_solutions_dict[tower_height] = solution_list
    
    stop = time()
    solution_times[tower_height] = stop - start
    print('tower_height: ', tower_height)
    print('time (minutes):', solution_times[tower_height] / 60)
    
final_counts = {th:len(sols) for th, sols in full_solutions_dict.items()}
print(final_counts)    

In [16]:
tower_height = 3
tower_width = 12

start = time()

mat = construct_dlx_matrix(tower_height, tower_width, 'U')
#write_dlx_matrix_to_txt('tower_two.txt', mat)
solver = DancingLinks(mat)
row_list_solutions = solver.solve()
print('total solutions: ', len(row_list_solutions))

stop = time()
solution_times[tower_height] = stop - start
print('time (minutes): ', solution_times[2] / 60)

KeyboardInterrupt: 